In [1]:
from TCGame_Env_GAME import TicTacToe
import collections
import numpy as np
import random
import pickle
import collections
from matplotlib import pyplot as plt
import time
import os
from datetime import datetime
import ast

In [2]:
env = TicTacToe()
current_game_state = env.state.copy()

In [3]:
# load model - Q_dict #
with open("../Agent To Agent Competition/odd_Agent_20190226_145335/odd_Q_Dict_Model_20190226_150428.pickle","rb") as f:
    Q_dict = pickle.load(f)

In [4]:
def Q_state(state):
    return ('-'.join(str(e) for e in state)).replace('nan','x')

In [5]:
def get_agent_action(game_state,q_dict_ref,random_select=False):
    if random_select:
        proposed_action = random.choice([ac for itr,ac in enumerate(env.action_space(game_state)[0])])
    else:
        proposed_action = max(q_dict_ref[Q_state(game_state)],key=q_dict_ref[Q_state(game_state)].get)
    return proposed_action

In [6]:
def execute_action(game_state,action,from_user=True):
    # action = (position,number)
    agent_allowed, user_allowed = env.action_space(game_state)
    if from_user and action in user_allowed:
        return env.state_transition(game_state,action)
    elif not from_user and action in agent_allowed:
        return env.state_transition(game_state,action)
    else:
        raise ValueError("Invalid input : %s , from_user : %s" %(str(action),str(from_user)))

In [7]:
def valid_actions(state):

    valid_Actions = []
    
    valid_Actions = [i for i in env.action_space(state)[0]]
    return valid_Actions

def add_to_dict(state,q_dict_ref):
    state1 = Q_state(state)
    
    valid_act = valid_actions(state)
    if state1 not in q_dict_ref.keys():
        print("State = %s was seen by agent for the first time :-0, adding it for learning" %(state1))
        for action in valid_act:
            q_dict_ref[state1][action]=0
    
    return q_dict_ref

#### (Testing) ####

In [8]:
Q_dict[Q_state(env.state)]

{(0, 1): -1.7793120816908399,
 (0, 3): -1.7795132605128308,
 (0, 5): -1.7797471147748856,
 (0, 7): -1.7797636472619942,
 (0, 9): -1.779371353423175,
 (1, 1): -1.7791723060375089,
 (1, 3): -1.779101777870136,
 (1, 5): -1.7792403689720222,
 (1, 7): -1.7792784235627197,
 (1, 9): -1.7792276197599057,
 (2, 1): -1.779310406650113,
 (2, 3): -1.7794176904429535,
 (2, 5): -1.7797983543831868,
 (2, 7): -1.7792356197351098,
 (2, 9): -1.7790743463153418,
 (3, 1): -1.7790889041923859,
 (3, 3): -1.779370679172739,
 (3, 5): -1.7794700960320782,
 (3, 7): -1.7790751953529171,
 (3, 9): -1.779117774752209,
 (4, 1): -1.7792588504670928,
 (4, 3): -1.7794829756487602,
 (4, 5): -1.7793453325458113,
 (4, 7): -1.7795189479536877,
 (4, 9): -1.779140963870029,
 (5, 1): -1.7795534564518114,
 (5, 3): -1.779249782461423,
 (5, 5): -1.7792227689559212,
 (5, 7): -1.7792124044439985,
 (5, 9): -1.7794397593561853,
 (6, 1): -1.779479997158969,
 (6, 3): -1.779472456318031,
 (6, 5): -1.779064462923111,
 (6, 7): -1.77917527

In [9]:
#print(env.state)
env.game_step(env.state,get_agent_action(env.state,Q_dict))

Agent Action : Position = 6, number = 5
After agent action, state
    ['X' 'X' 'X']
    ['X' 'X' 'X']
    ['5' 'X' 'X']
Give user action (position,number):(4,6)
After user input : Position = 4, number = 6
    ['X' 'X' 'X']
    ['X' '6' 'X']
    ['5' 'X' 'X']


([nan, nan, nan, nan, 6, nan, 5, nan, nan], -1, False, 'Continue')

## Just Play the game #

In [12]:
current_game_state = env.state.copy()
game_ended = False
Q_dict_copy = Q_dict.copy()
while not game_ended:
    agent_action = get_agent_action(current_game_state,Q_dict_copy)
    next_state, reward, game_ended, msg = env.game_step(current_game_state,agent_action)
    current_game_state = next_state
print(msg)
print(env.print_board(current_game_state))

Agent Action : Position = 0, number = 9
After agent action, state
    ['9' 'X' 'X']
    ['X' 'X' 'X']
    ['X' 'X' 'X']
Give user action (position,number):(4,2)
After user input : Position = 4, number = 2
    ['9' 'X' 'X']
    ['X' '2' 'X']
    ['X' 'X' 'X']
Agent Action : Position = 8, number = 3
After agent action, state
    ['9' 'X' 'X']
    ['X' '2' 'X']
    ['X' 'X' '3']
Give user action (position,number):(7,8)
After user input : Position = 7, number = 8
    ['9' 'X' 'X']
    ['X' '2' 'X']
    ['X' '8' '3']
Agent Action : Position = 1, number = 5
After agent action, state
    ['9' '5' 'X']
    ['X' '2' 'X']
    ['X' '8' '3']
Agent Won
    ['9' '5' 'X']
    ['X' '2' 'X']
    ['X' '8' '3']


### Play Game and Also Learn

In [22]:
def save_object(obj,filename):
    with open(filename,"wb") as f:
        pickle.dump(obj,f,pickle.HIGHEST_PROTOCOL)

In [ ]:
LR = 0.09
GAMMA = 0.99

In [94]:
current_game_state = env.state.copy()
game_ended = False
Q_dict_copy = Q_dict.copy()
while not game_ended:
    agent_action = get_agent_action(current_game_state,Q_dict_copy)
    next_state, reward, game_ended, msg = env.game_step(current_game_state,agent_action)
    
    Q_dict_copy = add_to_dict(next_state,Q_dict_copy.copy())
    
    if game_ended:
            best_next_action = None
            # Remember in this case, the Q_value for next state will be 0, hence we do (reward + GAMMA* 0)
            print("For Agent (Before) : when in state = %s, with action = %s, Q_value = %f" %(Q_state(current_game_state),str(agent_action),Q_dict_copy[Q_state(current_game_state)][agent_action]))
            Q_dict_copy[Q_state(current_game_state)][agent_action] += LR * ((reward + (GAMMA*0)) - Q_dict_copy[Q_state(current_game_state)][agent_action]) 
            print("For Agent (After) : when in state = %s, with action = %s, Q_value = %f" %(Q_state(current_game_state),str(agent_action),Q_dict_copy[Q_state(current_game_state)][agent_action]))
    else:
        # we update the current_state,agent_action reward based on "BEST" action possible for the next state #
        # we get the BEST ACTION for next_state and use that with the discount factor #
        best_next_action = max(Q_dict_copy[Q_state(next_state)],key=Q_dict_copy[Q_state(next_state)].get)
        print("For Agent (Before) : when in state = %s, with action = %s, Q_value = %f" %(Q_state(current_game_state),str(agent_action),Q_dict_copy[Q_state(current_game_state)][agent_action]))
        Q_dict_copy[Q_state(current_game_state)][agent_action] += LR * ((reward + (GAMMA*(Q_dict_copy[Q_state(next_state)][best_next_action]))) - Q_dict_copy[Q_state(current_game_state)][agent_action] ) 
        print("For Agent (After) : when in state = %s, with action = %s, Q_value = %f" %(Q_state(current_game_state),str(agent_action),Q_dict_copy[Q_state(current_game_state)][agent_action]))
    current_game_state = next_state
print(msg)
print(env.print_board(current_game_state))
Q_dict = Q_dict_copy.copy()
#filename = "Q_Policy_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".pickle.pk"
#save_object(Q_dict,filename)

Agent Action : Position = 3, number = 3
After agent action, state
    ['X' 'X' 'X']
    ['3' 'X' 'X']
    ['X' 'X' 'X']
Give user action (position,number):(4,8)
After user input : Position = 4, number = 8
    ['X' 'X' 'X']
    ['3' '8' 'X']
    ['X' 'X' 'X']
For Agent (Before) : when in state = x-x-x-x-x-x-x-x-x, with action = (3, 3), Q_value = -1.374272
For Agent (After) : when in state = x-x-x-x-x-x-x-x-x, with action = (3, 3), Q_value = -1.374086
Agent Action : Position = 5, number = 5
After agent action, state
    ['X' 'X' 'X']
    ['3' '8' '5']
    ['X' 'X' 'X']
Give user action (position,number):(8,4)
After user input : Position = 8, number = 4
    ['X' 'X' 'X']
    ['3' '8' '5']
    ['X' 'X' '4']
For Agent (Before) : when in state = x-x-x-3-8-x-x-x-x, with action = (5, 5), Q_value = -0.375968
For Agent (After) : when in state = x-x-x-3-8-x-x-x-x, with action = (5, 5), Q_value = -0.432131
Agent Action : Position = 7, number = 9
After agent action, state
    ['X' 'X' 'X']
    ['3'

In [95]:
save_object(Q_dict,"Q_policy_TicTactoe_201902015_143001.pickle,pkl")

In [75]:
Q_dict["x-x-x-x-x-x-x-x-x"]

{(0, 1): -1.389461173046447,
 (0, 3): -1.3848897055748177,
 (0, 5): -1.1978579030536172,
 (0, 7): -1.3823387339632986,
 (0, 9): -0.08788708562547459,
 (1, 1): -1.3301509818778756,
 (1, 3): -1.3785077007916595,
 (1, 5): -0.09170300651423155,
 (1, 7): -1.3826571147113114,
 (1, 9): -1.3249402828129628,
 (2, 1): -1.3799988003737713,
 (2, 3): -1.3331438174255488,
 (2, 5): -0.8827370140452909,
 (2, 7): -1.3533192778143368,
 (2, 9): -1.0147217639815886,
 (3, 1): -1.3312328667194415,
 (3, 3): -1.3742721122115256,
 (3, 5): -1.2253514205959806,
 (3, 7): -1.368011451340596,
 (3, 9): -1.3139866134873774,
 (4, 1): -1.3855284147408626,
 (4, 3): -1.3833060201196665,
 (4, 5): -1.3386942315203538,
 (4, 7): -1.3859401724507294,
 (4, 9): -1.1140268501381447,
 (5, 1): -1.3744107070159663,
 (5, 3): -1.3492899060777384,
 (5, 5): -1.3900035053030109,
 (5, 7): -1.3908355801047179,
 (5, 9): -1.3069316909883097,
 (6, 1): -1.3612513129501687,
 (6, 3): -1.3955129594397437,
 (6, 5): -1.3081785599808584,
 (6, 7): -

In [20]:
Q_dict["9-x-x-x-4-x-x-x-x"]

{(1, 1): -0.047736420512356326,
 (1, 3): -2.791825571023639,
 (1, 5): -1.5225140215398645,
 (1, 7): -1.251409390109311,
 (2, 1): -0.0714491543887863,
 (2, 3): -1.9928409649568601,
 (2, 5): -0.4724634542874121,
 (2, 7): -0.8886108018724314,
 (3, 1): -1.3615342281965954,
 (3, 3): -1.9461107673442282,
 (3, 5): -2.4972480588000643,
 (3, 7): -0.3976677329550965,
 (5, 1): -1.2252162936838946,
 (5, 3): -1.9854617263917105,
 (5, 5): -2.41059689500214,
 (5, 7): -1.4829769377497106,
 (6, 1): -0.5599883934190423,
 (6, 3): -1.4819395197183716,
 (6, 5): 0.09616074752446535,
 (6, 7): -1.04886615354119,
 (7, 1): -1.0658677498176283,
 (7, 3): -1.1490417497891972,
 (7, 5): -1.09300684283041,
 (7, 7): -1.0672945865830934,
 (8, 1): -0.5331701875857696,
 (8, 3): -0.9112574787707827,
 (8, 5): 0.8392769549866919,
 (8, 7): -0.9339828818482201}